# Digital diktanalyse: Mønstre i mengden

_18.februar 2026_

<!-- Ankomst: introduksjon -->
Velkommen til kodeverksted på Nasjonalbiblioteket! 

<!--Presentasjon: hvem er jeg? Hva er konteksten? -->

<!--
Jeg heter Ingerid Løyning Dale, og jobber som språkteknolog i Språkbanken og DH-laben her på Nasjonalbiblioteket. 
Kodebiblioteket vi kommer til å gå gjennom i dag heter `poetry_analysis` og ble utviklet i samarbeid med Ranveig Kvinnsland fra Ibsensenteret på UiO, i et forskningsprosjekt som heter NORN. Det er et EU-finansiert prosjekt som undersøker nasjonalromantikken i litteraturhistorien. 

Vi har sett mest på dikt fra 1890-tallet, altså den såkalt nyromantiske perioden, da vi ikke skrev på nynorsk og bokmål, men landsmål og riksmål, som fortsatt var veldig likt dansk. 
Men det betyr ikke at dere er låst til 1800-tallsdiktning! Koden fungerer på all tekst der linjeskift er brukt for å skille mellom elementene som skal sammenlignes. Vi deler altså ikke teksten inn i setninger, men i linjer, og ser på gjentakelser på tvers av disse linjene. 
-->

<!-- Agenda -->
## Agenda
I løpet av dette verkstedet skal vi annotere ulike gjentakelsesmønstre i digitale tekster med regelbaserte algoritmer.

Vi skal gå gjennom tre forskjellige lyriske virkemidler som baserer seg på gjentakelser: 
1. [Enderim](#enderim)
2. [Bokstavrim](#bokstavrim) 
3. [Anafor](#anafor)

Med egne funksjoner for hvert av trekkene kan du blant annet undersøke om det ofte er de samme ordene som rimer på hverandre i enderimsmønstre, om bokstavrim dannes med vokaler eller konsonanter,  og hvilke ord som oftest brukes anaforisk.

<!-- Presentasjon: formål med verktøyet vi bruker, vise frem appen, forklare hvilke lyriske trekk vi ser på og hvordan disse defineres -->

## App 
I tillegg til kodebiblioteket, som kan brukes av alle med en pythonversjon høyere enn 3.11, har vi  utviklet en egen app der man kan eksperimentere med enkelttekster og visualisere annotasjonene av gjentakelsesmønstrene: 

https://dh.nb.no/run/diktanalyse/app/

<!--
Som eksempeldikt til demoen har jeg valgt ut et dikt som passer måneden vi er inne i:  
-->
### Eksempeldikt
> "Februarvise" fra *Høstblomster eller rimstubber* (1896), av Andreas Aabel.

In [5]:
from pathlib import Path

filepath = Path("Februarvise.txt")
text = filepath.read_text()

print(text)

Jeg elsker Februar. 
Om hun kan være knipsk og morsk, 
hun er saa egte norsk. 
aar snøen lyser borti li – 
dikkom, dakkom, dikkom, dikkom dakk – 
da ut paa kjælke, ut paa ski! 
Heiopsan, her er vi! 


Hun klæder sig med glans. 
Ja hun er vinterns bedste pryd 
og allerstørste fryd. 
Kun tumler sig paa fjord og fjell – 
dikkom, dakkom, dikkom, dikkom dakk! – 
i rennefok og rykemjell; 
men det er moro lell! 


Av alle systre tolv 
er dig det mindste livsløp undt, 
og rapp din dag gaar rundt. 
men ei vi sakner Solens guld – 
dikkom, dakkom, dikkom, dikkom dakk! – 
naar bare Maanen er os huld, 
helst om han er – lidt fuld. 


Og i din klare natt, 
naar ingen stor, men mange smaa 
sig tér paa himlens blaa, 
da tindrer vel med deiligt skjær – 
dikkom, dakkom, dikkom, dikkom dakk! – 
din endeløse stjernehær 
og tykkes mig saa nær. 


Min egen Februar! 
Lad andre vælge sig April 
med dens troløse smil! 
Jeg vælger dig, og til din pris – 
dikkom, dakkom, dikkom, dikkom dakk! – 
jeg synger, mens 

## Enderim

<!-- Enderim: 
- Operasjonalisering
- fallgruver
- hva er et nødrim?
- Er det rim når samme ordet gjentas?
- Utfordringer: lydlikhet vs. tekstlikhet, sammensatte ord 
- Hvilke ord rimer oftest?  
-->

Enderim definerer vi som stavelsesrim på slutten av en verselinje, altså at siste stavelse på én verselinje rimer på siste stavelse i en annen verselinje. 


I praksis er koden implementert slik:
- Vi går gjennom alle verselinjene i en strofe eller et dikt 
- For hver linje starter vi på bakerste bokstav, og sammenligner bokstav for bokstav, med siste ordet på alle tidligere linjer i strofen
- Dersom det er overlapp mellom to ord, og den delen som overlapper inneholder en vokal, så sier vi at ordene rimer.
- Da annoterer vi disse linjene med en merkelapp fra alfabetet.
- Sånn får vi mønstre som ABBA, AABB og ABAB i strofer på fire linjer der to og to av linjene rimer. 

In [10]:
from poetry_analysis.rhyme_detection import tag_poem_file # type: ignore

# Annoter diktet i tekstfilen med funksjonen "tag_poem_file"
annotations = tag_poem_file(filepath, write_to_file=True)


Resultatet er en liste med annotasjoner for hver strofe, som igjen inneholder en liste med annotasjoner per linje. 

Ved å flate ut den hierarkiske datastrukturen kan vi  laste inn annotasjonene i en dataramme og se nærmere på dem: 

In [33]:
import pandas as pd

# Flat ut listen med annotasjoner og last inn i en dataramme
flattened_data = []
for stanza in annotations:
    stanza_id = stanza['stanza_id']
    rhyme_scheme = stanza['rhyme_scheme']
    for verse in stanza['verses']:
        flat_dict = verse.copy()
        # Disse to linjene bare rydder opp litt og fjerner noen felter vi ikke trenger
        del flat_dict["transcription"]
        del flat_dict["syllables"]
        # Legg til informasjon om strofen på hver av verselinjene
        flat_dict.update({
            'stanza_id': stanza_id,
            'rhyme_scheme': rhyme_scheme,
        })
        flattened_data.append(flat_dict)

# Lag en pandas dataramme
df = pd.DataFrame(flattened_data)
df

,rhyme_score,rhyme_tag,text,tokens,last_token,rhymes_with,verse_id,stanza_id,rhyme_scheme
0,0.0,a,Jeg elsker Februar.,"[jeg, elsker, februar]",februar,NaN,0,0,abbcdcc
1,0.0,b,"Om hun kan være knipsk og morsk,","[om, hun, kan, være, knipsk, og, morsk]",morsk,NaN,1,0,abbcdcc
2,1.0,b,hun er saa egte norsk.,"[hun, er, saa, egte, norsk]",norsk,1.0,2,0,abbcdcc
3,0.0,c,aar snøen lyser borti li –,"[aar, snøen, lyser, borti, li]",li,NaN,3,0,abbcdcc
4,0.0,d,"dikkom, dakkom, dikkom, dikkom dakk –","[dikkom, dakkom, dikkom, dikkom, dakk]",dakk,NaN,4,0,abbcdcc
5,1.0,c,"da ut paa kjælke, ut paa ski!","[da, ut, paa, kjælke, ut, paa, ski]",ski,3.0,5,0,abbcdcc
6,1.0,c,"Heiopsan, her er vi!","[heiopsan, her, er, vi]",vi,5.0,6,0,abbcdcc
7,0.0,a,Hun klæder sig med glans.,"[hun, klæder, sig, med, glans]",glans,NaN,0,1,abbcdcc
8,0.0,b,Ja hun er vinterns bedste pryd,"[ja, hun, er, vinterns, bedste, pryd]",pryd,NaN,1,1,abbcdcc
9,1.0,b,og allerstørste fryd.,"[og, allerstørste, fryd]",fryd,1.0,2,1,abbcdcc


## Bokstavrim

## Anafor